# 实例：股票数据分析案例

## DataFrame的组合与聚合以及透视表生成


Pandas 中常用的组合和聚合方法（透视表）

- groupby() / apply() / agg()
- stack() / unstack()
- melted()
- pivot_table()

透视表（pivot table）是用来汇总其它表的数据。 首先把源表分组（grouping），然后对各组内数据做汇总操作如排序、平均、累加、计数或字符串连接等。 透视表用于数据处理，在数据可视化程序如电子表格或商业智能软件中常见。 

HINT: 透视表的分析是针对类别型字段而言的

下面我们通过一个综合一点的数据分析案例，巩固上面学过的操作，同时练习一下dataframe的组合、聚合以及透视表生成的操作。

- 数据案例：

This data extracted from Yahoo! Finance consists of stock prices both for MSFT from year 2003 and for AAPL from 1984-09-07 to 2008-10-14.

> - Open 当日开盘价 
> - High 当日最高价
> - Low 当日最低价
> - Close 当日收盘价
> - Volume 当日成交量
> - Adj Close 已调整收盘价

首先导入股票的样本数据，msft.csv文件是微软的股价数据，aapl.csv是苹果的股价数据。

In [65]:
import numpy as np
import pandas as pd


msft = pd.read_csv("msft.csv", index_col=0, parse_dates=True)
aapl = pd.read_csv("aapl.csv", index_col=0, parse_dates=True)

msft.sort_index(inplace=True)
aapl.sort_index(inplace=True)

msft.shape, aapl.shape

/tmp/ipykernel_8464/1990654139.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  msft = pd.read_csv("msft.csv", index_col=0, parse_dates=True)


((65, 6), (6081, 6))

查看导入后数据的头部数据，默认输出前5行。

In [22]:
aapl.head()

,Open,High,Low,Close,Volume,Adj Close
Date,,,,,,
1984-09-07,26.50,26.87,26.25,26.50,2981600,3.02
1984-09-10,26.50,26.62,25.87,26.37,2346400,3.01
1984-09-11,26.62,27.37,26.62,26.87,5444000,3.07
1984-09-12,26.87,27.00,26.12,26.12,4773600,2.98
1984-09-13,27.50,27.62,27.50,27.50,7429600,3.14


In [23]:
msft.head()

,Open,High,Low,Close,Volume,Adj. Close*
Date,,,,,,
2003-06-19,26.09,26.39,26.01,26.07,63626900,25.92
2003-06-20,26.34,26.38,26.01,26.33,86048896,26.18
2003-06-23,26.14,26.24,25.49,25.78,52584500,25.64
2003-06-24,25.65,26.04,25.52,25.70,51820300,25.56
2003-06-25,25.64,25.99,25.14,25.26,60483500,25.12


看一下数据的时间跨度信息

In [24]:
aapl.index.min(), aapl.index.max(), msft.index.min(), msft.index.max()

(Timestamp('1984-09-07 00:00:00'),
 Timestamp('2008-10-14 00:00:00'),
 Timestamp('2003-06-19 00:00:00'),
 Timestamp('2003-09-19 00:00:00'))

检查一下导入的数据情况

In [25]:
aapl.info(), msft.info();

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6081 entries, 1984-09-07 to 2008-10-14
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       6081 non-null   float64
 1   High       6081 non-null   float64
 2   Low        6081 non-null   float64
 3   Close      6081 non-null   float64
 4   Volume     6081 non-null   int64  
 5   Adj Close  6081 non-null   float64
dtypes: float64(5), int64(1)
memory usage: 332.6 KB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 65 entries, 2003-06-19 to 2003-09-19
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Open         65 non-null     float64
 1   High         65 non-null     float64
 2   Low          65 non-null     float64
 3   Close        65 non-null     float64
 4   Volume       65 non-null     int64  
 5   Adj. Close*  65 non-null     float64
dtypes: float64(5), int64(1)
memory usage: 3.6

根据时间字段的灵活切片

In [26]:
aapl.loc['2003-09'][['Open', 'High', 'Low']].head()

,Open,High,Low
Date,,,
2003-09-02,22.66,22.90,22.40
2003-09-03,22.80,23.32,22.76
2003-09-04,23.16,23.25,22.77
2003-09-05,22.73,23.15,22.41
2003-09-08,22.48,22.79,22.47


取出微软公司的2003年8月和9月的股价数据切片，取出苹果公司2003年9月份的股价数据切片。

In [27]:
msftOC08 = msft.loc['2003-08'][['Open', 'Close']]
msftOC09 = msft.loc['2003-09'][['Open', 'Close']]

aaplOC09 = aapl.loc['2003-09'][['Open', 'Close']]

In [28]:
msftOC08.head()

,Open,Close
Date,,
2003-08-01,26.33,26.17
2003-08-04,26.15,26.18
2003-08-05,26.31,25.66
2003-08-06,25.54,25.65
2003-08-07,25.72,25.71


将微软公司的8月和9月数据合并

In [29]:
pd.concat([msftOC08.head(), msftOC09.head()])

,Open,Close
Date,,
2003-08-01,26.33,26.17
2003-08-04,26.15,26.18
2003-08-05,26.31,25.66
2003-08-06,25.54,25.65
2003-08-07,25.72,25.71
2003-09-02,26.70,27.26
2003-09-03,27.42,28.30
2003-09-04,28.10,28.43
2003-09-05,28.23,28.38


将微软和苹果的9月数据合并（各取前5条），这时你会发现index索引项不唯一了。按照某个日期取出的数据不唯一，且无法区分是哪个股票的。

In [34]:
pd.concat([msftOC09.head(), aaplOC09.head()]) # index索引项不唯一

,Open,Close
Date,,
2003-09-02,26.70,27.26
2003-09-03,27.42,28.30
2003-09-04,28.10,28.43
2003-09-05,28.23,28.38
2003-09-08,28.39,28.84
2003-09-02,22.66,22.85
2003-09-03,22.80,22.95
2003-09-04,23.16,22.83
2003-09-05,22.73,22.50


In [35]:
# duplicated index labels!
_.loc['2003-09-02']

,Open,Close
Date,,
2003-09-02,26.70,27.26
2003-09-02,22.66,22.85


这时，如果想区分该怎么办呢？可以通过指定“keys”参数来实现（无脑的 concat 开始变聪明）

In [36]:
pd.concat([msftOC09.head(), aaplOC09.head()], keys = ['MSFT', 'APPLE'])

Open  Close
      Date                    
MSFT  2003-09-02  26.70  27.26
      2003-09-03  27.42  28.30
      2003-09-04  28.10  28.43
      2003-09-05  28.23  28.38
      2003-09-08  28.39  28.84
APPLE 2003-09-02  22.66  22.85
      2003-09-03  22.80  22.95
      2003-09-04  23.16  22.83
      2003-09-05  22.73  22.50
      2003-09-08  22.48  22.74

同理，纵向也可以合并。通过指定“axis”参数来实现。

In [37]:
pd.concat([msftOC09.head(), aaplOC09.head()], axis=1)

,Open,Close,Open,Close
Date,,,,
2003-09-02,26.70,27.26,22.66,22.85
2003-09-03,27.42,28.30,22.80,22.95
2003-09-04,28.10,28.43,23.16,22.83
2003-09-05,28.23,28.38,22.73,22.50
2003-09-08,28.39,28.84,22.48,22.74


也同理，纵向也可以通过指定“keys”参数，来区分出level

In [38]:
pd.concat([msftOC09.head(), aaplOC09.head()], axis=1, keys = ['MSFT', 'APPLE'])

MSFT         APPLE       
             Open  Close   Open  Close
Date                                  
2003-09-02  26.70  27.26  22.66  22.85
2003-09-03  27.42  28.30  22.80  22.95
2003-09-04  28.10  28.43  23.16  22.83
2003-09-05  28.23  28.38  22.73  22.50
2003-09-08  28.39  28.84  22.48  22.74

下面我们再来熟悉一下使用merge的合并操作

In [39]:
msftOCR09 = msftOC09.reset_index()
aaplOCR09 = aapl.loc['2003-09'][['Volume']].reset_index()
msftOCR09.shape, aaplOCR09.shape

((14, 3), (21, 2))

In [40]:
msftOCR09

,Date,Open,Close
0,2003-09-02,26.70,27.26
1,2003-09-03,27.42,28.30
2,2003-09-04,28.10,28.43
3,2003-09-05,28.23,28.38
4,2003-09-08,28.39,28.84
5,2003-09-09,28.65,28.37
6,2003-09-10,28.03,27.55
7,2003-09-11,27.66,27.84
8,2003-09-12,27.48,28.34
9,2003-09-15,28.37,28.36


In [41]:
aaplOCR09

,Date,Volume
0,2003-09-02,8647600
1,2003-09-03,9601000
2,2003-09-04,7135000
3,2003-09-05,8576200
4,2003-09-08,5973000
5,2003-09-09,6441800
6,2003-09-10,8031800
7,2003-09-11,7631600
8,2003-09-12,6428200
9,2003-09-15,8101600


执行合并操作，并思考一下合并结果为什么是14行数据？

In [42]:
pd.merge(msftOCR09, aaplOCR09, )

,Date,Open,Close,Volume
0,2003-09-02,26.70,27.26,8647600
1,2003-09-03,27.42,28.30,9601000
2,2003-09-04,28.10,28.43,7135000
3,2003-09-05,28.23,28.38,8576200
4,2003-09-08,28.39,28.84,5973000
5,2003-09-09,28.65,28.37,6441800
6,2003-09-10,28.03,27.55,8031800
7,2003-09-11,27.66,27.84,7631600
8,2003-09-12,27.48,28.34,6428200
9,2003-09-15,28.37,28.36,8101600


merge函数可以指定更多参数来实现各种不同的合并方式。how是inner内连接，on是指定用于连接的字段名。

In [43]:
pd.merge(msftOCR09, aaplOCR09, on='Date', how='inner')

,Date,Open,Close,Volume
0,2003-09-02,26.70,27.26,8647600
1,2003-09-03,27.42,28.30,9601000
2,2003-09-04,28.10,28.43,7135000
3,2003-09-05,28.23,28.38,8576200
4,2003-09-08,28.39,28.84,5973000
5,2003-09-09,28.65,28.37,6441800
6,2003-09-10,28.03,27.55,8031800
7,2003-09-11,27.66,27.84,7631600
8,2003-09-12,27.48,28.34,6428200
9,2003-09-15,28.37,28.36,8101600


再看一下how=outer外连接的合并结果，可以对比观察一下内外连接的不同。

In [44]:
pd.merge(msftOCR09, aaplOCR09, on='Date', how='outer')

,Date,Open,Close,Volume
0,2003-09-02,26.70,27.26,8647600
1,2003-09-03,27.42,28.30,9601000
2,2003-09-04,28.10,28.43,7135000
3,2003-09-05,28.23,28.38,8576200
4,2003-09-08,28.39,28.84,5973000
5,2003-09-09,28.65,28.37,6441800
6,2003-09-10,28.03,27.55,8031800
7,2003-09-11,27.66,27.84,7631600
8,2003-09-12,27.48,28.34,6428200
9,2003-09-15,28.37,28.36,8101600


**透视表**
* 也许大多数人都有在Excel中使用数据透视表的经历，其实Pandas也提供了一个类似的功能，名为pivot_table。
* 将一个DataFrame的记录数据整合成表格(类似Excel中的数据透视表功能)，而且是按照pivot(‘index=xx’,’columns=xx’,’values=xx’)来整合的。

为了进行透视表的操作，我们先做些数据准备：
* 插入一个名为“Symbol”的字段分别标识不同数据的股票代码；
* 合并两只股票的8、9月份数据，并按索引值排序；
* 重置合并数据的索引；

In [45]:
msft.insert(0, 'Symbol', 'MSFT',)
aapl.insert(0, 'Symbol', 'AAPL')

combined = pd.concat([msft['2003-08':'2003-09'][['Symbol','Open', 'Close']], 
                      aapl['2003-08':'2003-09'][['Symbol','Open', 'Close']]]).sort_index()

s4p = combined.reset_index();

In [46]:
msft

,Symbol,Open,High,Low,Close,Volume,Adj. Close*
Date,,,,,,,
2003-06-19,MSFT,26.09,26.39,26.01,26.07,63626900,25.92
2003-06-20,MSFT,26.34,26.38,26.01,26.33,86048896,26.18
2003-06-23,MSFT,26.14,26.24,25.49,25.78,52584500,25.64
2003-06-24,MSFT,25.65,26.04,25.52,25.70,51820300,25.56
2003-06-25,MSFT,25.64,25.99,25.14,25.26,60483500,25.12
...,...,...,...,...,...,...,...
2003-09-15,MSFT,28.37,28.61,28.33,28.36,41432300,28.20
2003-09-16,MSFT,28.41,28.95,28.32,28.90,52060600,28.74
2003-09-17,MSFT,28.76,28.95,28.47,28.50,47221600,28.34


In [47]:
aapl

,Symbol,Open,High,Low,Close,Volume,Adj Close
Date,,,,,,,
1984-09-07,AAPL,26.50,26.87,26.25,26.50,2981600,3.02
1984-09-10,AAPL,26.50,26.62,25.87,26.37,2346400,3.01
1984-09-11,AAPL,26.62,27.37,26.62,26.87,5444000,3.07
1984-09-12,AAPL,26.87,27.00,26.12,26.12,4773600,2.98
1984-09-13,AAPL,27.50,27.62,27.50,27.50,7429600,3.14
...,...,...,...,...,...,...,...
2008-10-08,AAPL,85.91,96.33,85.68,89.79,78847900,89.79
2008-10-09,AAPL,93.35,95.80,86.60,88.74,57763700,88.74
2008-10-10,AAPL,85.70,100.00,85.00,96.80,79260700,96.80


In [77]:
combined

,Symbol,Open,Close
Date,,,
2003-08-01,MSFT,26.33,26.17
2003-08-01,AAPL,21.00,20.73
2003-08-04,MSFT,26.15,26.18
2003-08-04,AAPL,20.53,21.21
2003-08-05,MSFT,26.31,25.66
...,...,...,...
2003-09-24,AAPL,22.21,21.32
2003-09-25,AAPL,21.34,20.43
2003-09-26,AAPL,20.30,20.69


In [48]:
s4p

,Date,Symbol,Open,Close
0,2003-08-01,MSFT,26.33,26.17
1,2003-08-01,AAPL,21.00,20.73
2,2003-08-04,MSFT,26.15,26.18
3,2003-08-04,AAPL,20.53,21.21
4,2003-08-05,MSFT,26.31,25.66
...,...,...,...,...
72,2003-09-24,AAPL,22.21,21.32
73,2003-09-25,AAPL,21.34,20.43
74,2003-09-26,AAPL,20.30,20.69
75,2003-09-29,AAPL,21.49,21.30


基于上面的数据，制作生成一张简单的数据透视表

In [49]:
closes = s4p.pivot(index='Date', columns='Symbol', values='Close')
closes

Symbol,AAPL,MSFT
Date,,
2003-08-01,20.73,26.17
2003-08-04,21.21,26.18
2003-08-05,20.38,25.66
2003-08-06,19.63,25.65
2003-08-07,19.93,25.71
2003-08-08,19.64,25.58
2003-08-11,19.66,25.61
2003-08-12,19.70,25.73
2003-08-13,20.18,25.60


**堆叠**

DataFrame 在操作上和 pivot 功能类似的函数还有： .stack() 和 .unstack(). 这个函数可以进一步将 column 转为新一层级的 index。


In [50]:
stackedCloses = closes.stack()

stackedCloses

Date        Symbol
2003-08-01  AAPL      20.73
            MSFT      26.17
2003-08-04  AAPL      21.21
            MSFT      26.18
2003-08-05  AAPL      20.38
                      ...  
2003-09-24  AAPL      21.32
2003-09-25  AAPL      20.43
2003-09-26  AAPL      20.69
2003-09-29  AAPL      21.30
2003-09-30  AAPL      20.72
Length: 77, dtype: float64

再来看看 stacked 的数据表如何切片？

In [51]:
stackedCloses.index
stackedCloses['2003-09-02']['AAPL']

22.85

stacked 的数据又该如何 unstacked 回来？

In [52]:
unstackedCloses = stackedCloses.unstack()

unstackedCloses

Symbol,AAPL,MSFT
Date,,
2003-08-01,20.73,26.17
2003-08-04,21.21,26.18
2003-08-05,20.38,25.66
2003-08-06,19.63,25.65
2003-08-07,19.93,25.71
2003-08-08,19.64,25.58
2003-08-11,19.66,25.61
2003-08-12,19.70,25.73
2003-08-13,20.18,25.60


**melt() 方法**
* melt() 可以把一个 dataframe 转换成每一行都有一个唯一id标识的变量组合的数据格式
* 数据分析的时候经常要把宽数据--->>长数据（列转行），有点像我们用excel 做逆透视的过程

In [53]:
s4p

,Date,Symbol,Open,Close
0,2003-08-01,MSFT,26.33,26.17
1,2003-08-01,AAPL,21.00,20.73
2,2003-08-04,MSFT,26.15,26.18
3,2003-08-04,AAPL,20.53,21.21
4,2003-08-05,MSFT,26.31,25.66
...,...,...,...,...
72,2003-09-24,AAPL,22.21,21.32
73,2003-09-25,AAPL,21.34,20.43
74,2003-09-26,AAPL,20.30,20.69
75,2003-09-29,AAPL,21.49,21.30


In [54]:
pd.melt(s4p, id_vars=['Date', 'Symbol'])

,Date,Symbol,variable,value
0,2003-08-01,MSFT,Open,26.33
1,2003-08-01,AAPL,Open,21.00
2,2003-08-04,MSFT,Open,26.15
3,2003-08-04,AAPL,Open,20.53
4,2003-08-05,MSFT,Open,26.31
...,...,...,...,...
149,2003-09-24,AAPL,Close,21.32
150,2003-09-25,AAPL,Close,20.43
151,2003-09-26,AAPL,Close,20.69
152,2003-09-29,AAPL,Close,21.30


In [55]:
_[_.Date == '2003-09-03']

,Date,Symbol,variable,value
44,2003-09-03,MSFT,Open,27.42
45,2003-09-03,AAPL,Open,22.80
121,2003-09-03,MSFT,Close,28.30
122,2003-09-03,AAPL,Close,22.95


**分组**

* 在日常的数据分析中，经常需要将数据根据某个（多个）字段划分为不同的群体（group）进行分析，如电商领域将全国的总销售额根据省份进行划分，分析各省销售额的变化情况，社交领域将用户根据画像（性别、年龄）进行细分，研究用户的使用情况和偏好等;
* 在Pandas中，上述的数据处理操作主要运用groupby完成;
* 在对于一个dataframe调用了groupby()方法的结果不是实际的分组数据,而是会得到一个DataFrameGroupBy对象;


先准备分组练习的数据，重置索引

In [72]:
s4g = combined.reset_index()
s4g

,Date,Symbol,Open,Close
0,2003-08-01,MSFT,26.33,26.17
1,2003-08-01,AAPL,21.00,20.73
2,2003-08-04,MSFT,26.15,26.18
3,2003-08-04,AAPL,20.53,21.21
4,2003-08-05,MSFT,26.31,25.66
...,...,...,...,...
72,2003-09-24,AAPL,22.21,21.32
73,2003-09-25,AAPL,21.34,20.43
74,2003-09-26,AAPL,20.30,20.69
75,2003-09-29,AAPL,21.49,21.30


插入两列数据，分别是“年”和“月”：

In [73]:
s4g.insert(1, 'Year', pd.DatetimeIndex(s4g['Date']).year)
s4g.insert(2, 'Month',pd.DatetimeIndex(s4g['Date']).month)
s4g.drop('Date', axis=1, inplace=True)
s4g

,Year,Month,Symbol,Open,Close
0,2003,8,MSFT,26.33,26.17
1,2003,8,AAPL,21.00,20.73
2,2003,8,MSFT,26.15,26.18
3,2003,8,AAPL,20.53,21.21
4,2003,8,MSFT,26.31,25.66
...,...,...,...,...,...
72,2003,9,AAPL,22.21,21.32
73,2003,9,AAPL,21.34,20.43
74,2003,9,AAPL,20.30,20.69
75,2003,9,AAPL,21.49,21.30


In [74]:
grouped = s4g.groupby('Symbol')
grouped


In [75]:
type(grouped.groups)
# grouped.size()  # a nice summary of the size of all the groups

pandas.io.formats.printing.PrettyDict

In [76]:
grouped.groups

{'AAPL': [1, 3, 5, 6, 8, 10, 12, 14, 17, 18, 21, 23, 24, 27, 28, 30, 32, 35, 36, 38, 41, 43, 45, 47, 48, 51, 53, 54, 57, 58, 60, 62, 64, 67, 68, 70, 71, 72, 73, 74, 75, 76], 'MSFT': [0, 2, 4, 7, 9, 11, 13, 15, 16, 19, 20, 22, 25, 26, 29, 31, 33, 34, 37, 39, 40, 42, 44, 46, 49, 50, 52, 55, 56, 59, 61, 63, 65, 66, 69]}

In [77]:
for name, group in grouped:
    print(name)
    print(group.head())

AAPL
   Year  Month Symbol   Open  Close
1  2003      8   AAPL  21.00  20.73
3  2003      8   AAPL  20.53  21.21
5  2003      8   AAPL  21.35  20.38
6  2003      8   AAPL  20.06  19.63
8  2003      8   AAPL  19.73  19.93
MSFT
   Year  Month Symbol   Open  Close
0  2003      8   MSFT  26.33  26.17
2  2003      8   MSFT  26.15  26.18
4  2003      8   MSFT  26.31  25.66
7  2003      8   MSFT  25.54  25.65
9  2003      8   MSFT  25.72  25.71


总结来说，groupby的过程就是将原有的DataFrame按照groupby的字段（这里是“Symbol”），划分为若干个分组DataFrame，被分为多少个组就有多少个分组DataFrame。所以说，在groupby之后的一系列操作（如agg、apply等），均是基于子DataFrame的操作。理解了这点，也就基本摸清了Pandas中groupby操作的主要原理。

**聚合**

理解了分组之后，我们开始了解聚合 aggregate 。

聚合操作是groupby后非常常见的操作，会写SQL的朋友对此应该是非常熟悉了。聚合操作可以用来求和、均值、最大值、最小值等。

In [94]:
s4g.groupby(['Symbol']).agg('mean')

,Year,Month,Open,Close
Symbol,,,,
AAPL,2003.0,8.5,21.381905,21.389524
MSFT,2003.0,8.4,26.914000,27.009714


In [93]:
s4g.groupby(['Symbol'])[['Year', 'Close']].agg('mean')

,Year,Close
Symbol,,
AAPL,2003.0,21.389524
MSFT,2003.0,27.009714


In [92]:
s4g.groupby(['Symbol', 'Year', 'Month']).agg('mean')

Open      Close
Symbol Year Month                      
AAPL   2003 8      20.555714  20.626667
            9      22.208095  22.152381
MSFT   2003 8      26.073333  26.038571
            9      28.175000  28.466429

In [91]:
s4g.groupby(['Symbol', 'Year', 'Month']).agg('mean')

Open      Close
Symbol Year Month                      
AAPL   2003 8      20.555714  20.626667
            9      22.208095  22.152381
MSFT   2003 8      26.073333  26.038571
            9      28.175000  28.466429

In [95]:
s4g.groupby(['Symbol', 'Year', 'Month']).agg(['mean', 'std'])

Open                Close          
                        mean       std       mean       std
Symbol Year Month                                          
AAPL   2003 8      20.555714  0.706920  20.626667  0.864999
            9      22.208095  0.672805  22.152381  0.793057
MSFT   2003 8      26.073333  0.401115  26.038571  0.407852
            9      28.175000  0.725489  28.466429  0.701017

In [96]:
s4g.groupby(['Symbol', 'Year', 'Month']).agg(['mean', 'std']).unstack('Month')

Open                                     Close             \
                  mean                  std                 mean              
Month                8          9         8         9          8          9   
Symbol Year                                                                   
AAPL   2003  20.555714  22.208095  0.706920  0.672805  20.626667  22.152381   
MSFT   2003  26.073333  28.175000  0.401115  0.725489  26.038571  28.466429   

                                 
                  std            
Month               8         9  
Symbol Year                      
AAPL   2003  0.864999  0.793057  
MSFT   2003  0.407852  0.701017

综合本节所学的知识，思考一下：上面的透视表可以直接一个函数搞定么？

In [97]:
pd.pivot_table(s4g, index=['Symbol', 'Year'], values=['Open','Close'], columns=['Month'], aggfunc=['mean', 'std'])

mean                                        std            \
                 Close                  Open                Close             
Month                8          9          8          9         8         9   
Symbol Year                                                                   
AAPL   2003  20.626667  22.152381  20.555714  22.208095  0.864999  0.793057   
MSFT   2003  26.038571  28.466429  26.073333  28.175000  0.407852  0.701017   

                                 
                 Open            
Month               8         9  
Symbol Year                      
AAPL   2003  0.706920  0.672805  
MSFT   2003  0.401115  0.725489